In [2]:
from diffusers import DiffusionPipeline
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

repo_id = "Lykon/DreamShaper"
pipe = DiffusionPipeline.from_pretrained(repo_id)
pipe = pipe.to(device)


pipe.safety_checker = None



unet/diffusion_pytorch_model.safetensors not found


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

/home/tim.kuipers/mambaforge-pypy3/envs/diffusers_env/lib/python3.11/site-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(


In [3]:
img_URL = 'https://tonedeaf.thebrag.com/wp-content/uploads/2018/07/aphex-twin.jpg'
# download image
import requests
from PIL import Image
from io import BytesIO
in_image = Image.open(BytesIO(requests.get(img_URL).content))
print(f'Size: {in_image.size}')
in_image = in_image.resize((1024, 512))
# convert to torch tensor
import torchvision.transforms as transforms
preprocess = transforms.Compose([
    transforms.ToTensor(),
])
in_image = preprocess(in_image)
in_image = in_image.to(device)
print(f'Shape: {in_image.shape}')

Size: (970, 546)
Shape: torch.Size([3, 512, 1024])


In [4]:
from PIL import Image
import matplotlib.pyplot as plt
import torch
import torchvision.transforms as transforms
from IPython.display import display, clear_output
import time


'''
# Example: You can use torchvision transforms to convert the tensor to a format suitable for display
transform = transforms.ToPILImage()
image_pil = transform(torch.randn(1, 3, 512, 1024)[0])

# Display the initial image
plt.figure(figsize=(8, 8))
img_plot = plt.imshow(image_pil)
plt.axis('off')
plt.show()
'''

from torchvision.models.optical_flow import raft
from torchvision.models.optical_flow import Raft_Large_Weights
import torchvision.transforms.functional as vision_F

weights = Raft_Large_Weights.DEFAULT
flow_model = raft.raft_large(weights=weights, progress=False).to(device)
flow_model = flow_model.eval()

flow_stage_num = 1 # 0-11

transforms = weights.transforms()
def preprocess(img1_batch, img2_batch):
    img1_batch = vision_F.resize(img1_batch, size=[520, 960], antialias=False)
    img2_batch = vision_F.resize(img2_batch, size=[520, 960], antialias=False)
    return transforms(img1_batch.unsqueeze(0), img2_batch.unsqueeze(0))



def warp_image(image, flow):
    h, w, c = image.shape
    x = torch.arange(w, device=device).float().view(1, -1).expand(h, -1)
    y = torch.arange(h, device=device).float().view(-1, 1).expand(-1, w)
    grid = torch.stack([x, y], dim=2)
    new_grid = grid + flow
    x_new, y_new = new_grid[..., 0], new_grid[..., 1]
    # wrap x and y:
    # x_new = (x_new + w).fmod(w)
    # y_new = (y_new + h).fmod(h)
    x_new = x_new.clamp(1, w-1)
    y_new = y_new.clamp(1, h-1)
    x0, x1 = x_new.floor().long(), x_new.ceil().long()
    y0, y1 = y_new.floor().long(), y_new.ceil().long()
    
    Ia = image[y0, x0]
    Ib = image[y1, x0]
    Ic = image[y0, x1]
    Id = image[y1, x1]
    
    wa = (x1 - x_new).unsqueeze(2) * (y1 - y_new).unsqueeze(2)
    wb = (x1 - x_new).unsqueeze(2) * (y_new - y0).unsqueeze(2)
    wc = (x_new - x0).unsqueeze(2) * (y1 - y_new).unsqueeze(2)
    wd = (x_new - x0).unsqueeze(2) * (y_new - y0).unsqueeze(2)
    
    warped_image = wa * Ia + wb * Ib + wc * Ic + wd * Id
    return warped_image

fig, ax = plt.subplots(figsize=(8, 8))

prev_image = None
all_images = []

def cb(step: int, timestep: int, latents: torch.FloatTensor):
    global prev_image, all_images

    image = pipe.vae.decode(latents / pipe.vae.config.scaling_factor, return_dict=False)[0]
    image = pipe.image_processor.postprocess(image, output_type="pt", do_denormalize=None)
    
    if prev_image != None:
        for i in range(len(image)):
            image1 = prev_image[i]
            image2 = image[i]
            w, h = latents.shape[-2:]

            image1_p, image2_p = preprocess(image1, image2)
            with torch.no_grad():
                flow = flow_model(image1_p, image2_p)[flow_stage_num]
            flow = vision_F.resize(flow, size=[w, h], interpolation=Image.BILINEAR)[0]
            warped_latents = warp_image(latents[i].permute(1, 2, 0), flow.permute(1, 2, 0))

            ratio = timestep / 1000
            latents[i] = latents[i] * (1 - ratio) + ratio * warped_latents.permute(2, 0, 1)
    elif False:
        for i in range(len(image)):
            ratio = .1
            encoded = pipe.vae.tiled_encode(in_image.unsqueeze(0))[0].mean
            print(f'{encoded.shape}')
            latents[i] = latents[i] * (1- ratio) + ratio * encoded

    prev_image = image

    all_images.append(image)


prompt = 'alien, H.R. Giger, warp records'
out = pipe(prompt, 512, 1024, callback=cb)
out[0]

  0%|          | 0/50 [00:00<?, ?it/s]

/home/tim.kuipers/mambaforge-pypy3/envs/diffusers_env/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [ ]:
import torch
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import numpy as np
from IPython.display import HTML

# Create a torch tensor (batch_size, height, width, channels)
batch = [vision_F.resize(ims[0], size=[256, 512], antialias=False).permute(1, 2, 0).cpu().numpy() for ims in all_images]


fig, ax = plt.subplots()
im = ax.imshow(batch[0])

def update(frame):
    im.set_array(batch[frame])
    return [im]

ani = FuncAnimation(fig, update, frames=range(len(batch)), blit=True)
HTML(ani.to_jshtml())

NameError: name 'all_images' is not defined